In [ ]:
%pip install -U -q pandas pandasql openai Markdown2docx

In [40]:
import pandas as pd

with open("./output/data.xlsx", "rb") as f:
    df = pd.read_excel(f, sheet_name="tiger")

In [51]:
import json
import pandasql as ps

result_df = ps.sqldf("SELECT * FROM df WHERE adventure_name = 'Bobcat'", locals())

json_result = result_df.to_dict(orient="records")
activities = json.dumps(json_result)
activities = activities.encode().decode("unicode-escape")

# replace unicode characters...
activities = activities.replace("’", "'")
activities = activities.replace("“", '"')
activities = activities.replace("”", '"')

In [52]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.environ["OPENAI_KEY"],
)

chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": """You are a Cub Scouts den meeting planning expert. Using the activities data below, please plan a den meeting for the Tigers (1st graders).
It includes data on Energy Level (`activity_attr_energy`), how many supplies are required (`activity_attr_supplies`) and how long it takes (`activity_attr_time`). All three values range from 1 to 5, with 1 being lowest and 5 being highest.
Make sure to choose exactly 1 activity for each `requirement_name`. Try to choose higher energy levels with lower supplies and time requirements.
Include the following sections:
- Adventure name
- Adventure description
- Full list of supplies required for the meeting
- If there any at-home requirements for the meeting, note them
- Opening ceremony, including
    - Pledge of allegiance
    - Scout oath and law
- One section per requirement, including the chosen activity name, supplies and directions
            
Activities:
""",
        },
        {
            "role": "user",
            "content": activities,
        },
    ],
)

In [58]:
from Markdown2docx import Markdown2docx

plan = chat_completion.choices[0].message.content

with open("./output/plan.md", "w") as file:
    file.write(plan)

project = Markdown2docx("./output/plan")
project.eat_soup()
project.save()